# Handwritten Digit Classification on the MNIST Dataset


## Introduction

This project applies the universal workflow of machine learning from *Deep Learning with Python* (Chapter 4.5, 1st edition) to the MNIST handwritten digit dataset. The goal is to build, analyze, and improve a neural network that can recognize digits from images.

The task is to develop a machine learning model that can accurately classify images of handwritten digits (0–9) from the MNIST dataset. This is a supervised learning problem, where the model is trained on labeled examples of images and their corresponding digit classes. Each input is a 28×28 grayscale image, and the model outputs a label indicating which digit (0–9) it predicts for that image. The MNIST dataset is a widely used benchmark in machine learning and computer vision, consisting of 70,000 images in total: 60,000 for training and 10,000 for testing.

MNIST is an ideal choice for this project because it is relatively small, easy to load and preprocess, and has been extensively studied, which makes it well suited for learning and comparing different model configurations.

In this project, the models are restricted to Keras/TensorFlow Sequential architectures built only from Dense and Dropout layers, without using more complex layers such as convolutional layers. This constraint reflects the course requirements and encourages a focus on understanding the core ideas of fully connected neural networks and regularization, rather than relying on more advanced architectures.

The main metric for success in this project is classification accuracy on a held‑out test set, measuring the proportion of correctly classified digit images. The goal is to build a model that achieves high accuracy while following the DLWP universal workflow, rather than necessarily reaching state‑of‑the‑art performance. Additional metrics such as precision, recall, and F1‑score will be considered during evaluation to provide a more detailed picture of model performance, particularly if class imbalance or specific error types become relevant.

The workflow for this project follows the universal machine learning process described in Deep Learning with Python: defining the problem and dataset, choosing a success metric, and deciding on an evaluation protocol. After that, the data will be prepared and split, a baseline model will be built and evaluated, a larger model will be trained to intentionally overfit, and finally regularization techniques and hyperparameter tuning will be applied based on validation performance to improve generalization.

In this project, the universal workflow of machine learning from Deep Learning with Python is followed step by step. The problem and dataset are first defined, along with the success metric and evaluation protocol. The data is then loaded, preprocessed, and split into training, validation, and test sets. A simple baseline model is built and evaluated, followed by a larger model that is allowed to overfit in order to explore the model’s capacity. Finally, regularization techniques and hyperparameter tuning are applied to improve generalization, and the best model is evaluated on the test set and discussed in terms of its strengths, limitations, and possible extensions.

## 1. Problem definition and dataset

The problem addressed in this project is the classification of handwritten digits from the MNIST dataset. The MNIST dataset consists of 70,000 grayscale images of handwritten digits (0–9), each of size 28×28 pixels. It is split into a training set of 60,000 images and a test set of 10,000 images. This is a supervised multiclass classification problem, where the goal is to train a model that takes an image as input and outputs the correct digit label.

MNIST is a good choice for this project because it is widely used in the machine learning community, making it easy to compare results with existing work. It is also relatively small and straightforward to load and preprocess, which allows us to focus on the modeling aspects rather than on complex data handling. Additionally, the task of digit classification is a well‑defined and intuitive problem that serves as a gentle introduction to image classification and neural networks.

## 2. Measure of Success

Our mains metric for success in this project is classification accuracy on a held‑out test set, which measures the proportion of correctly classified digit images. 

$$
Accuracy = \frac{Number\ of\ Correct\ Predictions}{Total\ Number\ of\ Predictions}
$$

The goal is to build a model that achieves high accuracy while following the DLWP universal workflow, rather than necessarily reaching state‑of‑the‑art performance. Additional metrics such as precision, recall, and F1‑score will be considered during evaluation to provide a more detailed picture of model performance, particularly if class imbalance or specific error types become relevant.

## 3. Evaluation Protocol

The original MNIST dataset provides 60,000 training images and 10,000 test images. In this project, the 60,000 training images are further split into 50,000 training samples and 10,000 validation samples. The 50,000 training samples are used to fit the parameters of the neural network, while the 10,000 validation samples are used to monitor performance during development and to guide choices such as model architecture, number of epochs, and regularization strength.

The 10,000 test images are kept completely separate from the training and validation process and are only used once, at the end of the project, to obtain an unbiased estimate of the final model’s generalization performance.
This train/validation/test setup helps prevent overfitting to the test set and ensures that any improvements observed on the validation set reflect genuine improvements in the model, rather than adaptation to a single held‑out benchmark.


## 4. Data Loading and Preprocessing

MNIST dataset is available in Keras, which makes it easy to load and preprocess. The images are grayscale and have pixel values in the range [0, 255]. For better performance, we will normalize the pixel values to the range [0, 1] by dividing by 255. Additionally, since we are using a fully connected neural network, we will flatten the 28×28 images into 784‑dimensional vectors.

In [ ]:
import numpy as np
from tensorflow import keras

# 1) Load MNIST
(x_train, y_train), (x_test, y_test) = keras.datasets.mnist.load_data()
print(f"x_train shape: {x_train.shape}, y_train shape: {y_train.shape}")
print(f"x_test shape: {x_test.shape}, y_test shape: {y_test.shape}")

2026-02-22 14:46:45.762540: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2026-02-22 14:46:45.762786: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2026-02-22 14:46:45.800052: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2026-02-22 14:46:46.665817: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off,

x_train shape: (60000, 28, 28), y_train shape: (60000,)
x_test shape: (10000, 28, 28), y_test shape: (10000,)


To provide the model with the correct format for training, we will need to "flatten" the 28×28 images into 784‑dimensional vectors. This can be done using the `reshape` method in NumPy or by using the `Flatten` layer in Keras. This process transforms each 2D image into a 1D vector, which is the expected input format for a fully connected neural network.

In [2]:
# 2) Flatten images: (num_samples, 28, 28) -> (num_samples, 784)
num_train = x_train.shape[0]
num_test = x_test.shape[0]

x_train = x_train.reshape(num_train, 28 * 28)
x_test = x_test.reshape(num_test, 28 * 28)

print(f"x_train shape: {x_train.shape}, x_test shape: {x_test.shape}")

x_train shape: (60000, 784), x_test shape: (10000, 784)


Normalization of pixel values is crucial for training neural networks effectively, as it helps to ensure that the input data is on a similar scale, which can improve convergence during training. By dividing the pixel values by 255, we scale them to the range [0, 1], which is more suitable for the activation functions used in neural networks.

In [3]:
# 3) Convert to float and scale to [0, 1]
x_train = x_train.astype("float32") / 255.0
x_test = x_test.astype("float32") / 255.0
print(f"Pixel value range after normalization: [{x_train.min()}, {x_train.max()}]")

Pixel value range after normalization: [0.0, 1.0]


Next, we will convert the class labels (0–9) into one‑hot encoded vectors using `keras.utils.to_categorical`. This function takes a vector of class indices and returns a matrix of one‑hot encoded label vectors, which is the required format for training a neural network with categorical crossentropy loss.

In [4]:
# 4) One-hot encode labels: 0 -> [1,0,0,0,0,0,0,0,0,0], etc.
num_classes = 10

y_train_categorical = keras.utils.to_categorical(y_train, num_classes)
y_test_categorical = keras.utils.to_categorical(y_test, num_classes)

print(y_train.shape, y_train_categorical.shape)
print(y_test.shape, y_test_categorical.shape)


(60000,) (60000, 10)
(10000,) (10000, 10)


Validation set is created by splitting the original training set of 60,000 images into 50,000 training samples and 10,000 validation samples. The validation set is used to select model architectures and hyperparameters, while the separate 10,000‑image test set is used only once at the end to obtain an unbiased estimate of the final model’s performance.

In [5]:
# 5) Create training / validation split from the original training set
x_train_final = x_train[:50000]
y_train_final = y_train_categorical[:50000]

x_val = x_train[50000:]
y_val = y_train_categorical[50000:]

print(x_train_final.shape, y_train_final.shape)
print(x_val.shape, y_val.shape)
print(x_test.shape, y_test_categorical.shape)

(50000, 784) (50000, 10)
(10000, 784) (10000, 10)
(10000, 784) (10000, 10)


## 5. Baseline model: small dense network

The baseline model is a simple fully connected neural network with one hidden layer of 64 units and ReLU activation, followed by an output layer of 10 units with softmax activation for multiclass classification. This architecture is chosen for its simplicity and ability to learn non‑linear relationships in the data, while still being small enough to train quickly and serve as a useful baseline for comparison with more complex models.

We start by importing "layers" from Keras, which provides the building blocks for constructing our neural network. The `Dense` layer is used to create fully connected layers, the first one with 64 units and ReLU activation, and the second one with 10 units and softmax activation for outputting class probabilities. The `Sequential` model is used to stack these layers in a linear fashion.

Finally, we compile the model with the `RMSPROP` optimizer, categorical crossentropy loss (suitable for multiclass classification), and accuracy as the metric to monitor during training. This setup allows us to train the model effectively and evaluate its performance based on accuracy.

In [13]:
from keras import layers
from keras import models

model_baseline = keras.Sequential([
    layers.Dense(64, activation="relu", input_shape=(28 * 28,)),
    layers.Dense(10, activation="softmax")
])

model_baseline.compile(
    optimizer="rmsprop",
    loss="categorical_crossentropy",
    metrics=["accuracy"]
)

model_baseline.summary()

Model: "sequential_5"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense_10 (Dense)                │ (None, 64)             │        50,240 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_11 (Dense)                │ (None, 10)             │           650 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 50,890 (198.79 KB)

 Trainable params: 50,890 (198.79 KB)

 Non-trainable params: 0 (0.00 B)

Training the baseline and storing the history of training and validation accuracy allows us to analyze the model's learning process and identify potential issues such as underfitting or overfitting. By comparing the training and validation accuracy, we can gain insights into how well the model is generalizing to unseen data and whether it is learning meaningful patterns from the training set.

In [14]:
history_baseline = model_baseline.fit(
    x_train_final,
    y_train_final,
    epochs=5,
    batch_size=128,
    validation_data=(x_val, y_val)
)


Epoch 1/5
391/391 ━━━━━━━━━━━━━━━━━━━━ 2s 5ms/step - accuracy: 0.8860 - loss: 0.4229 - val_accuracy: 0.9382 - val_loss: 0.2354
Epoch 2/5
391/391 ━━━━━━━━━━━━━━━━━━━━ 2s 5ms/step - accuracy: 0.9380 - loss: 0.2151 - val_accuracy: 0.9535 - val_loss: 0.1739
Epoch 3/5
391/391 ━━━━━━━━━━━━━━━━━━━━ 2s 5ms/step - accuracy: 0.9517 - loss: 0.1686 - val_accuracy: 0.9587 - val_loss: 0.1486
Epoch 4/5
391/391 ━━━━━━━━━━━━━━━━━━━━ 2s 5ms/step - accuracy: 0.9591 - loss: 0.1398 - val_accuracy: 0.9630 - val_loss: 0.1335
Epoch 5/5
391/391 ━━━━━━━━━━━━━━━━━━━━ 2s 4ms/step - accuracy: 0.9656 - loss: 0.1197 - val_accuracy: 0.9651 - val_loss: 0.1243


Storing the training history also enables us to visualize the learning curves, which can help in diagnosing problems with the model and guiding further improvements. For example, if the training accuracy is high but the validation accuracy is low, it may indicate that the model is overfitting to the training data, suggesting that we may need to apply regularization techniques or gather more data. Conversely, if both training and validation accuracy are low, it may indicate underfitting, suggesting that we may need a more complex model or longer training time.

In [15]:
train_acc = history_baseline.history["accuracy"]
val_acc = history_baseline.history["val_accuracy"]

print("Training accuracy per epoch:", train_acc)
print("Validation accuracy per epoch:", val_acc)


Training accuracy per epoch: [0.8859999775886536, 0.9379799962043762, 0.9516800045967102, 0.95906001329422, 0.9655600190162659]
Validation accuracy per epoch: [0.9381999969482422, 0.953499972820282, 0.9587000012397766, 0.9629999995231628, 0.9650999903678894]


## 6. Overfitting model: larger dense network
lorem ipsum

## 7. Regularization and hyperparameter tuning

lorem ipsum

## 8. Final model evaluation on the test set

lorem ipsum

## 9. Discussion and conclusions

lorem ipsum

## 10. References and code credits

lorem ipsum